# Customer Churn Analysis
## Setting up the study
### Import libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.plotting import plot_lifetimes
import seaborn as sns
np.random.seed(42)

### Generate synthetic data

In [ ]:


n_customers = 20000

ages = np.random.randint(18, 70, size=n_customers)
monthly_spend = np.random.normal(60, 15, size=n_customers).clip(min=10).round(2)
contract_type = np.random.choice(['Month-to-Month', 'One_Year', 'Two_Year'], size=n_customers, p=[0.6, 0.3, 0.1])

contract_churn_prob = {
    'Month-to-Month': 0.5,
    'One_Year': 0.25,
    'Two_Year': 0.1
}
age_scaled = (ages - ages.min()) / (ages.max() - ages.min())

spend_scaled = (monthly_spend - monthly_spend.min()) / (monthly_spend.max() - monthly_spend.min())

churn_prob = np.array([
    contract_churn_prob[c] for c in contract_type
]) - 0.2 * age_scaled - 0.3 * spend_scaled

churn_prob = np.clip(churn_prob, 0.05, 0.95)

churned = np.random.binomial(1, churn_prob)

raw_tenure = np.random.exponential(scale=(15 - 10 * churned), size=n_customers)
tenure = np.clip(raw_tenure, a_min=1, a_max=36).astype(int)

data = pd.DataFrame({
    'tenure': tenure,
    'churned': churned,
    'age': ages,
    'monthly_spend': monthly_spend,
    'contract_type': contract_type
})

# Preview
data.head()

## Fitting and testing the model
### Kaplan Meier estimate of customer survival

In [ ]:
kmf = KaplanMeierFitter()

# Fit the model
kmf.fit(durations=data['tenure'], event_observed=data['churned'])

# Plot the survival function
plt.figure(figsize=(8,5))
kmf.plot()
plt.title('Kaplan-Meier Estimate of Customer Survival')
plt.xlabel('Tenure (Months)')
plt.ylabel('Probability of Retention')
plt.grid()
plt.show()

### Analyzing contract type as a covariate

In [ ]:

plt.figure(figsize=(10, 6))

for contract in data['contract_type'].unique():
    mask = data['contract_type'] == contract
    kmf.fit(durations=data[mask]['tenure'], event_observed=data[mask]['churned'], label=contract)
    kmf.plot_survival_function()

plt.title("Survival Function by Contract Type")
plt.xlabel("Tenure (Months)")
plt.ylabel("Probability of Retention")
plt.legend()
plt.grid()
plt.show()

### Summary of model

In [ ]:
data_encoded = pd.get_dummies(data, columns=['contract_type'], drop_first=True)

cph = CoxPHFitter()
cph.fit(data_encoded, duration_col='tenure', event_col='churned')

# Display summary
cph.print_summary()

### Hazard Ratios

In [ ]:
plt.figure(figsize=(8, 5))
cph.plot()
plt.title('Hazard Ratios from Cox Proportional Hazards Model')
plt.grid()
plt.show()

## Outcomes and business implications of the model
### Predicting Future Churn

In [ ]:

expected_cols = data_encoded.drop(columns=['tenure', 'churned']).columns

new_customers = pd.DataFrame([{
    'age': 30,
    'monthly_spend': 45,
    'contract_type_One Year': 0,
    'contract_type_Two Year': 0
},
{
    'age': 50,
    'monthly_spend': 80,
    'contract_type_One Year': 1,
    'contract_type_Two Year': 0
}])

for col in expected_cols:
    if col not in new_customers.columns:
        new_customers[col] = 0

new_customers = new_customers[expected_cols]

survival_funcs = cph.predict_survival_function(new_customers)

plt.figure(figsize=(8, 5))
for i, func in enumerate(survival_funcs.T.values):
    plt.step(survival_funcs.index, func, label=f'Customer {i+1}')
    
plt.title('Predicted Survival Functions for New Customers')
plt.xlabel('Time (Months)')
plt.ylabel('Probability of Retention')
plt.legend()
plt.grid()
plt.show()



### Predicted Revenue lost due to churn

In [ ]:

avg_monthly_spend = data['monthly_spend'].mean()

overall_survival = cph.predict_survival_function(data_encoded.drop(columns=['tenure', 'churned'])).mean(axis=1)

expected_loss = (1 - overall_survival) * avg_monthly_spend * len(data)

plt.figure(figsize=(9, 5))
plt.plot(expected_loss.index, expected_loss.values, color='red')
plt.title("Expected Cumulative Revenue Lost Over Time Due to Churn")
plt.xlabel("Time (Months)")
plt.ylabel("Revenue Lost ($)")
plt.grid()
plt.show()

### Predicted revenue gain from improving customer retention

In [ ]:

retention_gain = 0.2

original_survival = overall_survival

improved_survival = original_survival + retention_gain * (1 - original_survival)
improved_survival = improved_survival.clip(upper=1.0)  # Cap at 100% survival

improved_revenue = improved_survival * avg_monthly_spend * len(data)
original_revenue = original_survival * avg_monthly_spend * len(data)

revenue_saved = improved_revenue - original_revenue

plt.figure(figsize=(9, 5))
plt.plot(revenue_saved.index, revenue_saved.values, color='green', label=f'{int(retention_gain * 100)}% Retention Boost')
plt.title("Projected Revenue Saved Over Time by Improving Retention")
plt.xlabel("Time (Months)")
plt.ylabel("Revenue Saved ($)")
plt.legend()
plt.grid()
plt.show()


### Simulating customer retention programs

In [ ]:

young_threshold = 30
is_young = data['age'] <= young_threshold
is_monthly = data['contract_type'] == 'Month-to-Month'

high_risk = is_young & is_monthly

base_churn_rate = data['churned'].mean()

program_effectiveness = 0.30  # reduces churn by 30%
data_simulated = data.copy()

np.random.seed(42)
adjusted_churn_prob = np.where(
    high_risk,
    data_simulated['churned'] * (1 - program_effectiveness),
    data_simulated['churned']
)

data_simulated['churned_simulated'] = np.random.binomial(1, adjusted_churn_prob)

churn_before = data['churned'].mean()
churn_after = data_simulated['churned_simulated'].mean()
churn_reduction = churn_before - churn_after

print(f"Baseline churn rate: {churn_before:.2%}")
print(f"Post-program churn rate: {churn_after:.2%}")
print(f"Estimated churn reduction: {churn_reduction:.2%} overall")

import seaborn as sns

melted = pd.melt(data_simulated, 
                 id_vars=['age', 'contract_type'], 
                 value_vars=['churned', 'churned_simulated'], 
                 var_name='Churn Type', value_name='Churned')

plt.figure(figsize=(8,5))
sns.barplot(data=melted, x='Churn Type', y='Churned')
plt.title('Churn Rate Before vs After Retention Programs')
plt.ylabel('Churn Rate')
plt.show()


### Predicted revenue from customer retention programs

In [ ]:

def get_revenue_series(churned_flags, tenure_list, spend_list):
    revenue = np.zeros((n_customers, 36))  # up to 36 months
    for i in range(n_customers):
        months_active = tenure_list[i] if churned_flags[i] else 36
        months_active = min(months_active, 36)
        revenue[i, :months_active] = spend_list[i]
    return revenue

original_revenue_matrix = get_revenue_series(data['churned'], data['tenure'], data['monthly_spend'])
simulated_revenue_matrix = get_revenue_series(data_simulated['churned_simulated'], data['tenure'], data['monthly_spend'])

original_revenue_cumulative = original_revenue_matrix.sum(axis=0).cumsum()
simulated_revenue_cumulative = simulated_revenue_matrix.sum(axis=0).cumsum()

revenue_saved = simulated_revenue_cumulative - original_revenue_cumulative

plt.figure(figsize=(10, 6))
plt.plot(revenue_saved, color='green', label='Revenue Saved (Program Impact)')
plt.title("📈 Cumulative Revenue Saved Over Time from Retention Programs")
plt.xlabel("Time (Months)")
plt.ylabel("Revenue Saved ($)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


### Breakeven analysis

In [ ]:


program_cost_per_customer = 20  
num_customers_targeted = high_risk.sum()
total_program_cost = program_cost_per_customer * num_customers_targeted

breakeven_month = np.argmax(revenue_saved >= total_program_cost) if np.any(revenue_saved >= total_program_cost) else None


plt.figure(figsize=(10, 6))
plt.plot(revenue_saved, color='green', label='Revenue Saved (Program Impact)')
plt.axhline(total_program_cost, color='red', linestyle='--', label=f'Program Cost = ${total_program_cost}')
if breakeven_month is not None:
    plt.axvline(breakeven_month, color='blue', linestyle=':', label=f'Breakeven at Month {breakeven_month}')
    plt.text(breakeven_month+1, total_program_cost * 1.05, f'Month {breakeven_month}', color='blue')
else:
    plt.text(20, total_program_cost * 1.05, "No breakeven within 36 months", color='red')

plt.title("💸 Breakeven Analysis: Retention Program ROI")
plt.xlabel("Time (Months)")
plt.ylabel("Revenue Saved ($)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


print(f"📊 Total Program Cost: ${total_program_cost:,.2f}")
if breakeven_month is not None:
    print(f"✅ Breakeven reached at Month {breakeven_month} — program pays for itself.")
else:
    print("⚠️ Breakeven not reached within 36 months.")
